In [7]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv("final_dataset.csv")
print(df)

     Unnamed: 0    dunsNum leadergender        leader_race_expanded  \
0             0   58168600       female   Black or African-American   
1             1  605485861         male  White or European American   
2             2  783598485       female  White or European American   
3             3   20825064         male  White or European American   
4             4  148015845         male  White or European American   
..          ...        ...          ...                         ...   
242         242   79688478         male   Black or African-American   
243         243  619658904       female                       Asian   
244         244  787916092       female                       Asian   
245         245   73886782         male  White or European American   
246         246   70765601         male                       Asian   

    isWomanOwned             Label  
0            YES              None  
1             NO              None  
2            YES              None  

In [11]:
def flag_minorowneddesc(df):
    
    if (df['isWomanOwned'] == 'YES'):
        return 'Woman Leadership'
    elif (df['leader_race_expanded'] == 'American Indian or Alaska Native'):
        return 'Race : American Indian or Alaska Native Leadership'
    elif (df['leader_race_expanded'] == 'Asian'):
        return 'Race : Asian Leadership'
    elif (df['leader_race_expanded'] == 'Black or African-American'):
        return 'Race : Black or African-American Leadership'
    elif (df['leader_race_expanded'] == 'Hispanic or Latino'):
        return 'Race : Hispanic or Latino Leadership'
    elif (df['leader_race_expanded'] == 'Native Hawaiian or Pacific Islander'):
        return 'Race : Native Hawaiian or Pacific Islander'   
    elif (df['Label'] == 'LGBTQ'):
        return 'LGBTQ Leadership'
    elif (df['Label'] == 'DifferentlyAbled'):
        return 'DifferentlyAbled Leadership'
    elif (df['leader_race_expanded'] == 'White or European American'):
        return np.nan
    
df['minorowneddesc'] = df.apply(flag_minorowneddesc, axis = 1)

In [12]:
df

,Unnamed: 0,dunsNum,leadergender,leader_race_expanded,isWomanOwned,Label,minorowneddesc
0,0,58168600,female,Black or African-American,YES,None,Woman Leadership
1,1,605485861,male,White or European American,NO,None,NaN
2,2,783598485,female,White or European American,YES,None,Woman Leadership
3,3,20825064,male,White or European American,NO,None,NaN
4,4,148015845,male,White or European American,NO,LGBTQ,LGBTQ Leadership
...,...,...,...,...,...,...,...
242,242,79688478,male,Black or African-American,NO,None,Race : Black or African-American Leadership
243,243,619658904,female,Asian,YES,None,Woman Leadership
244,244,787916092,female,Asian,YES,DifferentlyAbled,Woman Leadership
245,245,73886782,male,White or European American,NO,DifferentlyAbled,DifferentlyAbled Leadership


## Import Azure Cosmos Libraries

In [89]:
import pandas as pd 
import json
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.errors as errors
import azure.cosmos.documents as documents
import azure.cosmos.http_constants as http_constants
from azure.cosmos.partition_key import PartitionKey
from pandas.io.json import json_normalize #package for flattening json in pandas df

print('Imported packages successfully.')

Imported packages successfully.


## Connect to Cosmos client

In [90]:
config = {
    "endpoint": "https://diversity-datastore.documents.azure.com:443/",
    "primarykey": "QDFFTPN60XYBGDElQxXe0RTY0QO8KZqjZj3mEFNsKkYonYgmvxpFi9W858279F7fm65IzjgrWWIAeaTXf2Wigg=="
}

client = cosmos_client.CosmosClient(url=config["endpoint"], credential={"masterKey":config["primarykey"]}
)


## Create Database in Azure Cosmos client

In [91]:
config = {
    "endpoint": "https://diversity-datastore.documents.azure.com:443/",
    "primarykey": "QDFFTPN60XYBGDElQxXe0RTY0QO8KZqjZj3mEFNsKkYonYgmvxpFi9W858279F7fm65IzjgrWWIAeaTXf2Wigg=="
}

client = cosmos_client.CosmosClient(url=config["endpoint"], credential={"masterKey":config["primarykey"]}
)

database_name = "diverseownedresult"
try:
    database = client.create_database(id=database_name)
except exceptions.CosmosResourceExistsError:
    database = client.get_database_client(database=database_name)

In [92]:
database_name = "diverseownedresult"


database = client.get_database_client(database_name)
CONTAINER_NAME = 'diverseownedresult'
try:
    container = database.create_container(id=CONTAINER_NAME, partition_key=PartitionKey(path="/dunsNum"),analytical_storage_ttl=-1)
except exceptions.CosmosResourceExistsError:
    container = database.get_container_client(CONTAINER_NAME)
except exceptions.CosmosHttpResponseError:
    raise

In [96]:
df.drop(['id'], axis = 1, inplace=True)

In [97]:
df

,dunsNum,leadergender,leader_race_expanded,isWomanOwned,Label,minorowneddesc
0,58168600,female,Black or African-American,YES,None,Woman Leadership
1,605485861,male,White or European American,NO,None,None
2,783598485,female,White or European American,YES,None,Woman Leadership
3,20825064,male,White or European American,NO,None,None
4,148015845,male,White or European American,NO,LGBTQ,LGBTQ Leadership
...,...,...,...,...,...,...
242,79688478,male,Black or African-American,NO,None,Race : Black or African-American Leadership
243,619658904,female,Asian,YES,None,Woman Leadership
244,787916092,female,Asian,YES,DifferentlyAbled,Woman Leadership
245,73886782,male,White or European American,NO,DifferentlyAbled,DifferentlyAbled Leadership


In [98]:

df['dunsNum']=df['dunsNum'].astype(str)

In [99]:
df

,dunsNum,leadergender,leader_race_expanded,isWomanOwned,Label,minorowneddesc
0,58168600,female,Black or African-American,YES,None,Woman Leadership
1,605485861,male,White or European American,NO,None,None
2,783598485,female,White or European American,YES,None,Woman Leadership
3,20825064,male,White or European American,NO,None,None
4,148015845,male,White or European American,NO,LGBTQ,LGBTQ Leadership
...,...,...,...,...,...,...
242,79688478,male,Black or African-American,NO,None,Race : Black or African-American Leadership
243,619658904,female,Asian,YES,None,Woman Leadership
244,787916092,female,Asian,YES,DifferentlyAbled,Woman Leadership
245,73886782,male,White or European American,NO,DifferentlyAbled,DifferentlyAbled Leadership


In [100]:
# Create Connection Link string
database_link = 'dbs/' + 'diverseownedresult'
collection_link = database_link + '/colls/' + 'diverseownedresult'

database_name = "diverseownedresult"


database = client.get_database_client(database_name)
CONTAINER_NAME = 'diverseownedresult'
container = database.get_container_client(CONTAINER_NAME)


# Write rows of a pandas DataFrame as items to the Database Container
for i in range(0,df.shape[0]):
    insert_data = container.upsert_item(dict(df.iloc[i,:])
    )
print('Records inserted successfully.')

Records inserted successfully.
